In [71]:
import spacy
import pandas as pd

In [72]:
# Import Medium Spacy File 
nlp = spacy.load('en_core_web_md')

In [73]:
# Input Files
recipes = pd.read_csv('4. FeasibleRecipes.csv')
useroutput = pd.read_csv('3. UserOutput.csv')

# Attribute Column
attributes = useroutput['attributes'].dropna()

In [90]:
# Create Threshold for Feasibility Scores
feasible_recipes = recipes[recipes['feasibility_score'] >= 0.6]

print(f"{len(feasible_recipes)} recipes that meet your requirements found!")

492 recipes that meet your requirements found!


In [91]:
# Spacy Similarity
doc1 = nlp(''.join(attributes))

# Initializing Similarity List 
spacy_scores = []

# Calculating Similarity
for text2 in feasible_recipes['review']:
    try:
        doc2 = nlp(text2)
        spacy_scores.append(doc1.similarity(doc2))
    except: 
        spacy_scores.append(0)

In [92]:
# Saving Review Similarity Score
feasible_recipes['Review Similarity'] = spacy_scores

/var/folders/j4/01bw0t0n5195l3682d9brqz40000gn/T/ipykernel_30402/2542721314.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feasible_recipes['Review Similarity'] = spacy_scores


In [131]:
# Evaluation Score
feasible_recipes['Evaluation Score'] = 0.7 *feasible_recipes['feasibility_score'] + 0.3 * feasible_recipes['Review Similarity']
feasible_recipes['Feasibility Score'] = feasible_recipes['feasibility_score']

/var/folders/j4/01bw0t0n5195l3682d9brqz40000gn/T/ipykernel_30402/3059707511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feasible_recipes['Evaluation Score'] = 0.7 *feasible_recipes['feasibility_score'] + 0.3 * feasible_recipes['Review Similarity']
/var/folders/j4/01bw0t0n5195l3682d9brqz40000gn/T/ipykernel_30402/3059707511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feasible_recipes['Feasibility Score'] = feasible_recipes['feasibility_score']


In [129]:
# Finding Recommendations
recommendations = feasible_recipes[['name', 'ingredients', 'cook_time', 'rating', 'category', 'url', 'Feasibility Score', 'Review Similarity', 'Evaluation Score']].groupby(['name', 'ingredients', 'cook_time', 'rating', 'category', 'url']).mean('Evaluation Score').sort_values('Evaluation Score', ascending = False)

In [148]:
recommendations.head()

,,,,,,Feasibility Score,Review Similarity,Evaluation Score
name,ingredients,cook_time,rating,category,url,,,
Smash Burger Taco,"['mayonnaise', 'ketchup', 'mustard', 'relish', 'hot sauce', 'lb ground round', 'kosher salt', 'ground black pepper', 'flour tortilla', 'american cheese cut half lengthwise', 'iceberg lettuce', 'onion']",20.0,4.6,Tacos,https://www.allrecipes.com/smash-burger-taco-recipe-7485747,0.75,0.583860,0.700158
Apple Cheddar Twists,"['apple', 'lemon juice', 'butter', 'brown sugar', 'puff pastry', 'extra sharp cheddar cheese', 'sea salt']",80.0,4.5,Christmas,https://www.allrecipes.com/apple-cheddar-twists-recipe-8709986,0.60,0.642528,0.612758
4-Ingredient Keto Peanut Butter Cookies,"['butter', 'lowcalorie natural sweetener', 'egg', 'sugarfree vanilla extract']",25.0,4.0,Sugar-Free Recipes,https://www.allrecipes.com/recipe/261181/4-ingredient-keto-peanut-butter-cookies/,0.60,0.610753,0.603226
Fried Egg Tortilla,"['butter', 'egg', 'taco seasoning taste', 'cheddar cheese', 'flour tortilla']",9.0,4.3,Cooking for One,https://www.allrecipes.com/fried-egg-tortilla-recipe-7561904,0.60,0.603685,0.601105
Applesauce,"['apple cored', 'water', 'ground cinnamon', 'ground clove', 'white sugar']",35.0,4.7,Applesauce,https://www.allrecipes.com/recipe/54346/applesauce/,0.60,0.587820,0.596346
